In [1]:
import os
import tensorflow as tf

2024-02-22 15:06:16.121992: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-22 15:06:16.161401: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-22 15:06:16.161428: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-22 15:06:16.162401: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-22 15:06:16.169328: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-22 15:06:16.170289: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
os.chdir("/home/sakshmeno/Documents/GitHub/C-RNN-approach/Labels")
# os.chdir("/Users/saksh.menon/Documents/GitHub/C-RNN-approach/Labels")
with open("DATASET.txt") as dataset_obj:
    codeLines = dataset_obj.read()

In [3]:
def code_preprocessing(file):
    with open(file) as dataset_obj:
        codeLines = dataset_obj.read()
    comment_lines = []
    raw_codeLines = codeLines.replace("\t","").split("\n")
    multi_line_flag = 0
    for line_number in range(len(raw_codeLines)):
        if multi_line_flag:
            if raw_codeLines[line_number].__contains__("*/"):
                multi_line_flag = 0
            comment_lines.append(line_number)
        elif raw_codeLines[line_number].__contains__("/*"):
            if raw_codeLines[line_number].startswith("/*") and not(raw_codeLines[line_number].__contains__("*/")):
                comment_lines.append(line_number)
                multi_line_flag = 1
            elif raw_codeLines[line_number].__contains__("/*") and not(raw_codeLines[line_number].startswith("/*")):
                if raw_codeLines[line_number].__contains__("*/"):
                    psuedo_multi_line_start = raw_codeLines[line_number].find("/*")
                    psuedo_multi_line_end = raw_codeLines[line_number].find("*/")
                    temporary_line = raw_codeLines[line_number][:psuedo_multi_line_start] + raw_codeLines[line_number][psuedo_multi_line_end+2:]
                    raw_codeLines[line_number] = temporary_line

        elif raw_codeLines[line_number].startswith("//"):
            comment_lines.append(line_number)
        elif raw_codeLines[line_number].__contains__("//"):
            comment_start = raw_codeLines[line_number].find('//')
            raw_codeLines[line_number] = raw_codeLines[line_number][:comment_start]
            
    comment_lines.reverse()
    for i in comment_lines:
        raw_codeLines.pop(i)

    def insert_space(string, index):
        string_copy = ""
        for i in range(len(string)):
            if i==(index):
                string_copy += " "
                string_copy += string[i]
                string_copy += " "
                continue
            string_copy += string[i]
        return string_copy

    def find_char_indices(input_string, char):
        indices = []
        replacement_token = 0
        for index, character in enumerate(input_string):
            if character == char:
                indices.append(index + 2*replacement_token)
                replacement_token+=1
        return indices

    def space_out(string, char):
        indices = find_char_indices(string, char)

        for i in indices:
            string = insert_space(string, i)
        return string

    # for line_number in range(len(raw_codeLines)):
    #     placeHolder = raw_codeLines[line_number]
        # placeHolder = space_out(placeHolder, ";")
        # placeHolder = space_out(placeHolder, "(")
        # placeHolder = space_out(placeHolder, ")")
        # placeHolder = space_out(placeHolder, ",")
        # if placeHolder.endswith(";"):
        #     raw_codeLines[line_number] = "<start> " + placeHolder.replace(";","<end>")
        # elif not(placeHolder.endswith(";")):
        #     raw_codeLines[line_number] = "<start> " + placeHolder + " <end>"

    return raw_codeLines


In [16]:
dataset = code_preprocessing('DATASET.txt')
f = open('FILTERED_DATASET.txt','w')
for i in range(len(dataset)):
    f.write(dataset[i])
    f.write('\n')

In [17]:
text_ds = tf.data.TextLineDataset("/home/sakshmeno/Documents/GitHub/C-RNN-approach/Labels/FILTERED_DATASET.txt").filter(lambda x: tf.cast(tf.strings.length(x), bool))

In [20]:
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  return tf.strings.regex_replace(lowercase,
                                  '[%s]' % re.escape(string.punctuation), '')


# Define the vocabulary size and the number of words in a sequence.
vocab_size = 4096
sequence_length = 10

# Use the `TextVectorization` layer to normalize, split, and map strings to
# integers. Set the `output_sequence_length` length to pad all samples to the
# same length.
vectorize_layer = tf.keras.layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

In [25]:
import re
vectorize_layer.adapt(text_ds.batch(1024))

In [26]:
# Save the created vocabulary for reference.
inverse_vocab = vectorize_layer.get_vocabulary()
print(inverse_vocab[:20])

['', '[UNK]', '0', 'if', '1', 'i', 'include', 'uint8t', '16', 'break', 'void', 'null', 'char', 'return', 'else', 'case', 'for', 'int', 'define', 'ret']


In [28]:
# Vectorize the data in text_ds.
SEED = 42
AUTOTUNE = tf.data.AUTOTUNE
text_vector_ds = text_ds.batch(1024).prefetch(AUTOTUNE).map(vectorize_layer).unbatch()

In [29]:
sequences = list(text_vector_ds.as_numpy_iterator())
print(len(sequences))

4527


In [32]:
for seq in sequences:
  print(f"{seq} => {[inverse_vocab[i] for i in seq]}")

[   6 2477    0    0    0    0    0    0    0    0] => ['include', 'compilerh', '', '', '', '', '', '', '', '']
[   6 1271    0    0    0    0    0    0    0    0] => ['include', 'timedelayh', '', '', '', '', '', '', '', '']
[   6 2073    0    0    0    0    0    0    0    0] => ['include', 'hardwareprofileh', '', '', '', '', '', '', '', '']
[   6 1639    0    0    0    0    0    0    0    0] => ['include', 'peripheralsnandh', '', '', '', '', '', '', '', '']
[   6 1642    0    0    0    0    0    0    0    0] => ['include', 'peripheralsaccelh', '', '', '', '', '', '', '', '']
[ 45 132   0   0   0   0   0   0   0   0] => ['ifdef', 'usegyro', '', '', '', '', '', '', '', '']
[   6 1640    0    0    0    0    0    0    0    0] => ['include', 'peripheralsgyroh', '', '', '', '', '', '', '', '']
[26  0  0  0  0  0  0  0  0  0] => ['endif', '', '', '', '', '', '', '', '', '']
[   6 1638    0    0    0    0    0    0    0    0] => ['include', 'peripheralsrtch', '', '', '', '', '', '', '', '']
[